In [114]:
import pandas as pd
import re

In [115]:
cat62 = pd.read_csv("data/cat-62.csv")

In [116]:

#cat62 = cat62[cat62["flightid"] != "Data final não disponível"]

In [117]:


cat62['dt_radar'] = pd.to_datetime(cat62['dt_radar'], unit='ms')

In [118]:
bimtra = pd.read_csv("data/bimtra.csv")
bimtra['dt_dep'] = pd.to_datetime(bimtra['dt_dep'], unit='ms')
bimtra['dt_arr'] = pd.to_datetime(bimtra['dt_arr'], unit='ms')

In [119]:
bimtra["target"] = (bimtra["dt_arr"] - bimtra["dt_dep"]) // pd.Timedelta('1s')

In [120]:
bimtra["linha"] = bimtra["origem"] + bimtra["destino"]

In [121]:
bimtra = bimtra.drop_duplicates(subset='flightid')

In [122]:
#bimtra = bimtra[bimtra["target"] > 1500]

In [123]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL
...,...,...,...,...,...,...,...
278879,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ
278880,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ
278882,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ


In [124]:
cat62 = cat62.merge(bimtra, on="flightid", how="left")

In [125]:
cat62["linha"] = cat62["origem"] + cat62["destino"]

In [126]:
cat62.sort_values(by=['flightid', 'dt_radar'], inplace=True)

In [127]:
from math import radians, sin, cos, sqrt, atan2, degrees

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # raio da Terra em km

    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [128]:


cat62["lat"] = cat62["lat"].apply(degrees)
cat62["lon"] = cat62["lon"].apply(degrees)

In [129]:
cat62[cat62["flightid"] == "0000f76a51b99be3f181ea2b201b34ef"]

,flightid,lat,lon,flightlevel,speed,dt_radar,origem,destino,dt_dep,dt_arr,target,linha
17732362,0000f76a51b99be3f181ea2b201b34ef,-12.898787,-38.277699,14.0,170.0,2022-11-23 09:21:00,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732677,0000f76a51b99be3f181ea2b201b34ef,-12.898186,-38.274271,15.0,171.0,2022-11-23 09:21:04,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732364,0000f76a51b99be3f181ea2b201b34ef,-12.916182,-38.222831,29.0,245.0,2022-11-23 09:22:01,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732653,0000f76a51b99be3f181ea2b201b34ef,-12.921176,-38.222295,31.0,245.0,2022-11-23 09:22:05,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732650,0000f76a51b99be3f181ea2b201b34ef,-12.985469,-38.252465,59.0,273.0,2022-11-23 09:23:01,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
...,...,...,...,...,...,...,...,...,...,...,...,...
17732632,0000f76a51b99be3f181ea2b201b34ef,-22.179636,-46.103180,250.0,391.0,2022-11-23 11:07:02,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732665,0000f76a51b99be3f181ea2b201b34ef,-22.184071,-46.107347,245.0,389.0,2022-11-23 11:07:06,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732699,0000f76a51b99be3f181ea2b201b34ef,-22.261151,-46.168817,233.0,384.0,2022-11-23 11:08:03,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR
17732643,0000f76a51b99be3f181ea2b201b34ef,-22.349540,-46.223459,219.0,374.0,2022-11-23 11:09:03,SBSV,SBGR,2022-11-23 09:20:26,2022-11-23 11:27:51,7645.0,SBSVSBGR


In [130]:
resumo_voo = cat62.groupby('flightid').agg({
    'dt_radar': lambda x: (x.max() - x.min()).total_seconds(),
    'lat': ['first', 'last'],
    'lon': ['first', 'last'],
    'flightlevel': ['mean', 'min', 'max', 'std'],
    'speed': ['mean', 'min', 'max', 'std'],
    'linha': 'first'
})


In [131]:
resumo_voo

dt_radar        lat                   lon  \
                                 <lambda>      first       last      first   
flightid                                                                     
0000f76a51b99be3f181ea2b201b34ef   6543.0 -12.898787 -22.453113 -38.277699   
00010bb2c49076d56a9815122fefc8aa   4083.0 -19.644736 -23.626155 -43.958119   
00010ed6d09c78b5a37f4df13f524f56   7740.0 -13.035493 -23.623237 -38.282166   
000143522a56fa245543e477c74fd908  10019.0 -23.609967  -8.135660 -46.567788   
00014fa436546dfdc7a67919e9e1ea47   4924.0 -19.641217 -27.365809 -43.961015   
...                                   ...        ...        ...        ...   
fffeba6342187a9ee0caff2526e4ff0e   4563.0 -23.578866 -29.996528 -46.689515   
ffff413d10cfbb298a9411855e80a0f0   6602.0 -22.874854 -29.994711 -43.126958   
ffff62189abb326d9f18ff1412e5de8e   3782.0 -19.630710 -23.437804 -43.970909   
ffffd0e5ce497de7ddf3d34d19b9ce3a   2581.0 -23.590285 -22.939142 -46.671998   
ffffd4331f8b78c6e726eda4e6a2f5ac   9482.0 -23.415394  -8.125157 -46.426775   

                                            flightlevel               \
                                       last        mean   min    max   
flightid                                                               
0000f76a51b99be3f181ea2b201b34ef -46.233340  308.194737  14.0  340.0   
00010bb2c49076d56a9815122fefc8aa -46.656405  214.356522  25.0  360.0   
00010ed6d09c78b5a37f4df13f524f56 -46.658587  318.598086  24.0  380.0   
000143522a56fa245543e477c74fd908 -34.923127  298.994382   1.0  370.0   
00014fa436546dfdc7a67919e9e1ea47 -48.217413  281.933824  26.0  360.0   
...                                     ...         ...   ...    ...   
fffeba6342187a9ee0caff2526e4ff0e -51.181826  263.000000   1.0  380.0   
ffff413d10cfbb298a9411855e80a0f0 -51.177427  305.910995   5.0  380.0   
ffff62189abb326d9f18ff1412e5de8e -46.493859  187.427273  26.0  360.0   
ffffd0e5ce497de7ddf3d34d19b9ce3a -43.155191  144.435897   3.0  270.0   
ffffd4331f8b78c6e726eda4e6a2f5ac -34.923833  318.339161   1.0  371.0   

                                                   speed                \
                                         std        mean    min    max   
flightid                                                                 
0000f76a51b99be3f181ea2b201b34ef   70.927036  409.636842  170.0  440.0   
00010bb2c49076d56a9815122fefc8aa  115.735496  312.017391  106.0  424.0   
00010ed6d09c78b5a37f4df13f524f56  106.012886  389.071770  145.0  436.0   
000143522a56fa245543e477c74fd908  118.987703  416.511236  123.0  477.0   
00014fa436546dfdc7a67919e9e1ea47  100.075024  393.022059  142.0  449.0   
...                                      ...         ...    ...    ...   
fffeba6342187a9ee0caff2526e4ff0e  129.199031  364.925926  122.0  433.0   
ffff413d10cfbb298a9411855e80a0f0  117.219396  335.778351  110.0  419.0   
ffff62189abb326d9f18ff1412e5de8e  126.294796  317.663636  142.0  421.0   
ffffd0e5ce497de7ddf3d34d19b9ce3a   93.353620  307.051282  120.0  372.0   
ffffd4331f8b78c6e726eda4e6a2f5ac  102.369508  433.730769  131.0  492.0   

                                                linha  
                                        std     first  
flightid                                               
0000f76a51b99be3f181ea2b201b34ef  42.689970  SBSVSBGR  
00010bb2c49076d56a9815122fefc8aa  94.817274  SBCFSBSP  
00010ed6d09c78b5a37f4df13f524f56  58.582779  SBSVSBSP  
000143522a56fa245543e477c74fd908  83.686207  SBSPSBRF  
00014fa436546dfdc7a67919e9e1ea47  63.645601  SBCFSBFL  
...                                     ...       ...  
fffeba6342187a9ee0caff2526e4ff0e  79.087701  SBSPSBPA  
ffff413d10cfbb298a9411855e80a0f0  58.055247  SBRJSBPA  
ffff62189abb326d9f18ff1412e5de8e  92.550435      None  
ffffd0e5ce497de7ddf3d34d19b9ce3a  73.091812  SBSPSBRJ  
ffffd4331f8b78c6e726eda4e6a2f5ac  73.327965  SBGRSBRF  

[278099 rows x 14 columns]

In [132]:
resumo_voo['distancia'] = resumo_voo.apply(lambda row: haversine(row[('lat', 'first')], row[('lon', 'first')], 
                                                           row[('lat', 'last')], row[('lon', 'last')]), axis=1)


In [133]:
# Aplicar a função join às colunas do DataFrame
resumo_voo.reset_index(inplace=True)


In [134]:
resumo_voo

flightid dt_radar        lat             \
                                         <lambda>      first       last   
0       0000f76a51b99be3f181ea2b201b34ef   6543.0 -12.898787 -22.453113   
1       00010bb2c49076d56a9815122fefc8aa   4083.0 -19.644736 -23.626155   
2       00010ed6d09c78b5a37f4df13f524f56   7740.0 -13.035493 -23.623237   
3       000143522a56fa245543e477c74fd908  10019.0 -23.609967  -8.135660   
4       00014fa436546dfdc7a67919e9e1ea47   4924.0 -19.641217 -27.365809   
...                                  ...      ...        ...        ...   
278094  fffeba6342187a9ee0caff2526e4ff0e   4563.0 -23.578866 -29.996528   
278095  ffff413d10cfbb298a9411855e80a0f0   6602.0 -22.874854 -29.994711   
278096  ffff62189abb326d9f18ff1412e5de8e   3782.0 -19.630710 -23.437804   
278097  ffffd0e5ce497de7ddf3d34d19b9ce3a   2581.0 -23.590285 -22.939142   
278098  ffffd4331f8b78c6e726eda4e6a2f5ac   9482.0 -23.415394  -8.125157   

              lon            flightlevel                                speed  \
            first       last        mean   min    max         std        mean   
0      -38.277699 -46.233340  308.194737  14.0  340.0   70.927036  409.636842   
1      -43.958119 -46.656405  214.356522  25.0  360.0  115.735496  312.017391   
2      -38.282166 -46.658587  318.598086  24.0  380.0  106.012886  389.071770   
3      -46.567788 -34.923127  298.994382   1.0  370.0  118.987703  416.511236   
4      -43.961015 -48.217413  281.933824  26.0  360.0  100.075024  393.022059   
...           ...        ...         ...   ...    ...         ...         ...   
278094 -46.689515 -51.181826  263.000000   1.0  380.0  129.199031  364.925926   
278095 -43.126958 -51.177427  305.910995   5.0  380.0  117.219396  335.778351   
278096 -43.970909 -46.493859  187.427273  26.0  360.0  126.294796  317.663636   
278097 -46.671998 -43.155191  144.435897   3.0  270.0   93.353620  307.051282   
278098 -46.426775 -34.923833  318.339161   1.0  371.0  102.369508  433.730769   

                                    linha    distancia  
          min    max        std     first               
0       170.0  440.0  42.689970  SBSVSBGR  1355.300518  
1       106.0  424.0  94.817274  SBCFSBSP   523.193228  
2       145.0  436.0  58.582779  SBSVSBSP  1471.285722  
3       123.0  477.0  83.686207  SBSPSBRF  2121.248922  
4       142.0  449.0  63.645601  SBCFSBFL   962.125338  
...       ...    ...        ...       ...          ...  
278094  122.0  433.0  79.087701  SBSPSBPA   841.245187  
278095  110.0  419.0  58.055247  SBRJSBPA  1125.897592  
278096  142.0  421.0  92.550435      None   497.260753  
278097  120.0  372.0  73.091812  SBSPSBRJ   366.466603  
278098  131.0  492.0  73.327965  SBGRSBRF  2096.255597  

[278099 rows x 16 columns]

In [135]:
resumo_voo = resumo_voo.T.reset_index().T

In [136]:
arr = []
for i in range(0, len(resumo_voo.iloc[0].values)):
    arr.append(resumo_voo.iloc[0].values[i] + "_" + resumo_voo.iloc[1].values[i])

In [137]:
resumo_voo = resumo_voo.rename(columns=dict(zip(resumo_voo.columns, arr)))


In [138]:
resumo_voo = resumo_voo.iloc[2:]

In [139]:
resumo_voo.columns

Index(['flightid_', 'dt_radar_<lambda>', 'lat_first', 'lat_last', 'lon_first',
       'lon_last', 'flightlevel_mean', 'flightlevel_min', 'flightlevel_max',
       'flightlevel_std', 'speed_mean', 'speed_min', 'speed_max', 'speed_std',
       'linha_first', 'distancia_'],
      dtype='object')

In [140]:
resumo = resumo_voo.groupby('linha_first').agg({
    'flightid_': 'count',
    'dt_radar_<lambda>': ['mean', 'min', 'max', 'std'],
    'flightlevel_mean': 'mean',
    'flightlevel_max': 'mean',
    'distancia_': 'mean',
    'flightlevel_std': ['std', 'mean'],
    'speed_std': ['mean','std']
})

# Exibir o resumo
print(resumo)

            flightid_ dt_radar_<lambda>                                \
                count              mean     min      max          std   
linha_first                                                             
SBBRSBCF         2644       3210.663767     0.0   8277.0   506.981575   
SBBRSBCT          959       5524.459854     4.0   8166.0   602.814715   
SBBRSBFL          315       6614.263492     4.0  14041.0   824.447364   
SBBRSBGL          639       4499.599374   362.0   7138.0   539.795848   
SBBRSBGR         3124       4384.066261   238.0  15064.0   636.873657   
...               ...               ...     ...      ...          ...   
SBSVSBKP         1379       7368.569978   121.0  11225.0   761.985305   
SBSVSBPA           79      11771.683544  1683.0  13623.0  1455.223285   
SBSVSBRF         1491        3784.31053     0.0   9182.0   526.762331   
SBSVSBRJ         1630       6706.908589   843.0  10924.0   630.301570   
SBSVSBSP         3519       7789.326229   305.0  15

In [141]:
resumo.reset_index(inplace=True)
resumo.to_csv("data/resumo_voo.csv", index=False)

In [142]:
import pandas as pd
resumo = pd.read_csv("data/resumo_voo.csv")
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,575.6958980433018,15.414484379289922,113.11099040127081,96.26012004571069,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,1061.908684010797,13.184672048521254,113.58039360634044,84.08358708763086,13.509180931157234
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,1282.3324062624968,15.232888859932347,115.89589711595092,79.03449466219763,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,885.994544233408,13.252080132619195,122.03779859280792,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,1426.4684886193365,17.341259072228155,102.26689170812047,68.5258508294501,16.681647582814303
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,2277.3825023041554,8.22824879734405,99.87921093160097,62.36285594137942,8.362795726130804
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,643.117568170169,15.707041969914366,127.34337283196844,97.04344098509462,14.56471447373063
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,1225.918106541467,12.702016763699218,123.87140594500069,82.84562463100339,13.215119734561947


In [143]:
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,575.6958980433018,15.414484379289922,113.11099040127081,96.26012004571069,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,1061.908684010797,13.184672048521254,113.58039360634044,84.08358708763086,13.509180931157234
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,1282.3324062624968,15.232888859932347,115.89589711595092,79.03449466219763,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,885.994544233408,13.252080132619195,122.03779859280792,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,1426.4684886193365,17.341259072228155,102.26689170812047,68.5258508294501,16.681647582814303
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,2277.3825023041554,8.22824879734405,99.87921093160097,62.36285594137942,8.362795726130804
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,643.117568170169,15.707041969914366,127.34337283196844,97.04344098509462,14.56471447373063
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,1225.918106541467,12.702016763699218,123.87140594500069,82.84562463100339,13.215119734561947


In [144]:
import pandas as pd
esperas = pd.read_csv("data/esperas.csv")
esperas["dt_dep"] = pd.to_datetime(esperas["hora"], unit='ms')
esperas["hora"] = pd.to_datetime(esperas["hora"], unit='ms')
esperas["diaSemana"] = esperas["hora"].dt.dayofweek

esperas["hora"] = esperas["hora"].dt.hour

In [145]:
esperas_hora = esperas.groupby(["aero", "hora"]).agg({"esperas": ["mean", "sum"]})
esperas_semana = esperas.groupby(["aero", "diaSemana"]).agg({"esperas": ["mean", "sum"]})

In [146]:

esperas_aero = esperas.groupby(["aero"]).agg({"esperas": ["mean", "sum"]})

In [147]:
esperas

,esperas,hora,aero,dt_dep,diaSemana
0,0,0,SBBR,2022-06-01 00:00:00,2
1,0,1,SBBR,2022-06-01 01:00:00,2
2,0,2,SBBR,2022-06-01 02:00:00,2
3,0,3,SBBR,2022-06-01 03:00:00,2
4,0,4,SBBR,2022-06-01 04:00:00,2
...,...,...,...,...,...
93307,0,19,SBSV,2023-04-15 19:00:00,5
93308,0,20,SBSV,2023-04-15 20:00:00,5
93309,0,21,SBSV,2023-04-15 21:00:00,5
93310,0,22,SBSV,2023-04-15 22:00:00,5


In [148]:
bimtra["diaSemana"] = bimtra["dt_dep"].dt.dayofweek

bimtra["hora"] = bimtra["dt_dep"].dt.hour

In [149]:
bimtra['HoraDataDest'] = bimtra['dt_arr'].dt.strftime('%Y-%m-%d %H')

In [150]:
#esperas.to_csv("data/esperas_join.csv", index=False)

In [151]:
bimtra.shape

(262416, 10)

In [152]:
bimtra['HoraData'] = bimtra['dt_dep'].dt.strftime('%Y-%m-%d %H')
esperas['HoraData'] = esperas['dt_dep'].dt.strftime('%Y-%m-%d %H')

# Mesclar os dataframes usando a coluna "HoraData"
df_merge = pd.merge(bimtra, esperas, on='HoraData', how='left')

# Preencher NaN na coluna de espera com 0 (sem espera)
df_merge['esperas'] = df_merge['esperas'].fillna(0)
df_merge = df_merge.groupby("flightid").agg({"esperas": "sum"})
bimtra = bimtra.merge(df_merge.reset_index(), on="flightid", how="left")

In [153]:
bimtra.shape

(262416, 12)

In [154]:
esperas_hora.columns = esperas_hora.columns.droplevel(0)

In [155]:
esperas_hora.reset_index(inplace=True)

In [156]:
esperas_semana.columns = esperas_semana.columns.droplevel(0)
esperas_semana.reset_index(inplace=True)

In [157]:
esperas_aero.columns = esperas_aero.columns.droplevel(0)
esperas_aero.reset_index(inplace=True)

In [158]:
esperas_aero

,aero,mean,sum
0,SBBR,0.024434,190
1,SBCF,0.150463,1170
2,SBCT,0.037809,294
3,SBFL,0.015947,124
4,SBGL,0.012860,100
5,SBGR,0.255401,1986
6,SBKP,0.191872,1492
7,SBPA,0.018904,147
8,SBRF,0.019033,148
9,SBRJ,0.256816,1997


In [159]:
esperas_semana

,aero,diaSemana,mean,sum
0,SBBR,0,0.009058,10
1,SBBR,1,0.019022,21
2,SBBR,2,0.034420,38
3,SBBR,3,0.022163,25
4,SBBR,4,0.069149,78
...,...,...,...,...
79,SBSV,2,0.011775,13
80,SBSV,3,0.015071,17
81,SBSV,4,0.006206,7
82,SBSV,5,0.003623,4


In [160]:
esperas_hora.to_csv("data/esperas_hora.csv", index=False)
esperas_semana.to_csv("data/esperas_semana.csv", index=False)

In [161]:
bimtra.shape

(262416, 12)

In [162]:
bimtra = bimtra.merge(esperas_hora, right_on=["aero", "hora"], left_on=["origem", "hora"] , how="left")
bimtra = bimtra.merge(esperas_semana, right_on=["aero", "diaSemana"], left_on=["origem", "diaSemana"] , how="left")

In [163]:
bimtra.shape

(262416, 18)

In [164]:
esperas_aero.to_csv("data/esperas_aero.csv", index=False)

In [165]:
bimtra.shape

(262416, 18)

In [166]:
bimtra = bimtra.merge(esperas_aero, right_on=["aero"], left_on=["origem"] , how="left")

In [167]:
bimtra.shape

(262416, 21)

In [168]:
#import pandas as pd
metar = pd.read_csv("data/metar.csv")

In [169]:
metar

,hora,metar,aero
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV


In [170]:
import pandas as pd
from metar.Metar import Metar

# Função para parsear METAR
def parse_metar(metar):
    metar = metar.replace('METAF', 'METAR') 
    try:
        metar_obj = Metar(metar)
        temperature = metar_obj.temp.value() if metar_obj.temp else None
        dewpoint = metar_obj.dewpt.value() if metar_obj.dewpt else None
        wind_speed = metar_obj.wind_speed.value() if metar_obj.wind_speed else None
        wind_direction = metar_obj.wind_dir.value() if metar_obj.wind_dir else None
        visibility = metar_obj.vis.value() if metar_obj.vis else None
        pressure = metar_obj.press.value() if metar_obj.press else None
        weather = ', '.join([str(c) for c in metar_obj.weather]) if metar_obj.weather else None
        sky = ', '.join([str(c) for c in metar_obj.sky]) if metar_obj.sky else None
        return pd.Series([temperature, dewpoint, wind_speed, wind_direction, visibility, pressure, weather, sky])
    except Exception as e:
        #print(str(e))
        return pd.Series([None, None, None, None, None, None, None, None])

# Aplicar parse sobre METAR para cada linha
metar[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metar['metar'].apply(parse_metar)


/tmp/ipykernel_73086/1735850452.py:23: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metar[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metar['metar'].apply(parse_metar)


In [171]:
metar

,hora,metar,aero,Temperatura,Ponto de Orvalho,Velocidade do Vento,Direção do Vento,Visibilidade,Pressão,Tempo,Céu
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR,21.0,8.0,2.0,70.0,10000.0,1018.0,NaN,NaN
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR,20.0,9.0,2.0,100.0,10000.0,1019.0,NaN,NaN
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,0.0,0.0,10000.0,1019.0,NaN,NaN
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,2.0,270.0,10000.0,1019.0,NaN,NaN
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR,14.0,9.0,0.0,0.0,10000.0,1018.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV,29.0,23.0,8.0,80.0,10000.0,1011.0,None,"('FEW', <metar.Datatypes.distance object at 0x..."
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV,29.0,23.0,7.0,90.0,10000.0,1011.0,None,NaN
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0,10000.0,1011.0,None,NaN
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0,10000.0,1011.0,None,NaN


In [172]:
metar.iloc[0]["metar"]

'METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018='

In [173]:

metar['HoraData'] = pd.to_datetime(metar['hora'], unit='ms').dt.strftime('%Y-%m-%d %H')
# Mesclar os dataframes usando a coluna "HoraData"



# Preencher NaN na coluna de espera com 0 (sem espera)


In [174]:
del bimtra["aero"]

In [175]:
# Agrupando por flight_id
df_merge = pd.merge(bimtra, metar, left_on=['destino', 'HoraDataDest'], right_on=['aero', 'HoraData'], how='left')

grouped_df = df_merge.groupby('flightid')

# Calculando as estatísticas
stats_df = grouped_df.agg({
    'Temperatura': ['mean', 'min', 'max', 'std'],
    'Ponto de Orvalho': ['mean', 'min', 'max', 'std'],
    'Velocidade do Vento': ['mean', 'min', 'max', 'std'],
    'Direção do Vento': ['mean', 'min', 'max', 'std'],
    'Visibilidade': ['mean', 'min', 'max', 'std'],
    'Pressão': ['mean', 'min', 'max', 'std']
})
stats_df.reset_index(inplace=True)
stats_df.columns = [''.join(col) for col in stats_df.columns]
bimtra = bimtra.merge(stats_df, on="flightid", how="left")

In [176]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraDataDest,...,Direção do Ventomax,Direção do Ventostd,Visibilidademean,Visibilidademin,Visibilidademax,Visibilidadestd,Pressãomean,Pressãomin,Pressãomax,Pressãostd
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 01,...,150.0,NaN,10000.0,10000.0,10000.0,NaN,1015.0,1015.0,1015.0,NaN
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 01,...,150.0,NaN,10000.0,10000.0,10000.0,NaN,1015.0,1015.0,1015.0,NaN
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 01,...,150.0,NaN,10000.0,10000.0,10000.0,NaN,1015.0,1015.0,1015.0,NaN
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 01,...,100.0,NaN,10000.0,10000.0,10000.0,NaN,1018.0,1018.0,1018.0,NaN
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 02,...,210.0,NaN,10000.0,10000.0,10000.0,NaN,1020.0,1020.0,1020.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,2023-04-15 22,...,180.0,NaN,10000.0,10000.0,10000.0,NaN,1014.0,1014.0,1014.0,NaN
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,2023-04-15 22,...,180.0,NaN,10000.0,10000.0,10000.0,NaN,1014.0,1014.0,1014.0,NaN
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,2023-04-15 22,...,180.0,NaN,10000.0,10000.0,10000.0,NaN,1014.0,1014.0,1014.0,NaN
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 14,...,20.0,NaN,10000.0,10000.0,10000.0,NaN,1011.0,1011.0,1011.0,NaN


In [177]:
import pandas as pd

metaf = pd.read_csv("data/metaf.csv")
metaf['metaf'].apply(parse_metar)
metaf['metaf'] = metaf['metaf'].apply(lambda x: ' '.join(x.split()))

metaf[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metaf['metaf'].apply(parse_metar)

metaf['HoraData'] = pd.to_datetime(metaf['hora'], unit='ms').dt.strftime('%Y-%m-%d %H')
# Mesclar os dataframes usando a coluna "HoraData"


/tmp/ipykernel_73086/2430131428.py:4: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metaf['metaf'].apply(parse_metar)
/tmp/ipykernel_73086/2430131428.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metaf[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metaf['metaf'].apply(parse_metar)


In [178]:

df_merge = pd.merge(bimtra, metaf, left_on=['destino', 'HoraDataDest'], right_on=['aero', 'HoraData'], how='left')
# Agrupando por flight_id
grouped_df = df_merge.groupby('flightid')

# Calculando as estatísticas
stats_df = grouped_df.agg({
    'Temperatura': ['mean', 'min', 'max', 'std'],
    'Ponto de Orvalho': ['mean', 'min', 'max', 'std'],
    'Velocidade do Vento': ['mean', 'min', 'max', 'std'],
    'Direção do Vento': ['mean', 'min', 'max', 'std'],
    'Visibilidade': ['mean', 'min', 'max', 'std'],
    'Pressão': ['mean', 'min', 'max', 'std']
})
stats_df.columns = [''.join(col) for col in stats_df.columns]
stats_df.reset_index(inplace=True)

bimtra = bimtra.merge(stats_df, on="flightid", how="left")
# Preencher NaN na coluna de espera com 0 (sem espera)
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraDataDest,...,Direção do Ventomax_y,Direção do Ventostd_y,Visibilidademean_y,Visibilidademin_y,Visibilidademax_y,Visibilidadestd_y,Pressãomean_y,Pressãomin_y,Pressãomax_y,Pressãostd_y
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,2023-04-15 22,...,170.0,NaN,6000.0,6000.0,6000.0,NaN,1014.0,1014.0,1014.0,NaN
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,2023-04-15 22,...,170.0,NaN,6000.0,6000.0,6000.0,NaN,1014.0,1014.0,1014.0,NaN
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,2023-04-15 22,...,170.0,NaN,6000.0,6000.0,6000.0,NaN,1014.0,1014.0,1014.0,NaN
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 14,...,230.0,NaN,10000.0,10000.0,10000.0,NaN,1013.0,1013.0,1013.0,NaN


In [179]:
tcprev = pd.read_csv("data/tc-prev.csv")

In [180]:
tcprev["hora"] = pd.to_datetime(tcprev["hora"], unit='ms')

In [181]:
tcprev

,hora,troca,aero
0,2022-06-01 00:00:00,0,BR
1,2022-06-01 01:00:00,0,BR
2,2022-06-01 02:00:00,0,BR
3,2022-06-01 03:00:00,0,BR
4,2022-06-01 04:00:00,0,BR
...,...,...,...
93307,2023-04-15 19:00:00,1,SV
93308,2023-04-15 20:00:00,1,SV
93309,2023-04-15 21:00:00,1,SV
93310,2023-04-15 22:00:00,0,SV


In [182]:
tcprev.to_csv("data/tc-prev-join.csv", index=False)

In [183]:
bimtra.shape

(262416, 68)

In [184]:
# Mesclar os dataframes usando a coluna "HoraData"
tcprev['HoraData'] = tcprev['hora'].dt.strftime('%Y-%m-%d %H')
df_merge = pd.merge(bimtra, tcprev, on='HoraData', how='left')

# Preencher NaN na coluna de espera com 0 (sem espera)
df_merge['troca'] = df_merge['troca'].fillna(0)
df_merge = df_merge.groupby("flightid").agg({"troca": "sum"})
bimtra = bimtra.merge(df_merge.reset_index(), on="flightid", how="left")

In [185]:
bimtra.shape

(262416, 69)

In [186]:
tcprev["diaSemana"] = tcprev["hora"].dt.dayofweek

tcprev["hora"] = tcprev["hora"].dt.hour
tcprev_hora = tcprev.groupby(["aero", "hora"]).agg({"troca": ["mean", "sum"]})
tcprev_sem = tcprev.groupby(["aero", "diaSemana"]).agg({"troca": ["mean", "sum"]})
tcprev_aero = tcprev.groupby(["aero"]).agg({"troca": ["mean", "sum"]})

In [187]:
tcreal = pd.read_csv("data/tc-real.csv")
tcreal["hora"] = pd.to_datetime(tcreal["hora"], unit='ms')
tcreal["diaSemana"] = tcreal["hora"].dt.dayofweek

tcreal['HoraData'] = tcreal['hora'].dt.strftime('%Y-%m-%d %H')
tcreal["hora"] = tcreal["hora"].dt.hour

In [188]:
#tcreal = pd.read_csv("data/tc-real.csv")
tcreal

,hora,nova_cabeceira,antiga_cabeceira,aero,diaSemana,HoraData
0,14,32,03,FL,2,2022-06-01 14
1,18,18,12,RF,2,2022-06-01 18
2,19,16,13,CF,2,2022-06-01 19
3,19,15,26,GL,2,2022-06-01 19
4,19,15,26,GL,2,2022-06-01 19
...,...,...,...,...,...,...
31694,19,15,30,KP,5,2023-04-15 19
31695,20,17,10,SV,5,2023-04-15 20
31696,20,10,17,SV,5,2023-04-15 20
31697,20,15,08,GL,5,2023-04-15 20


In [189]:
tcreal.to_csv("data/tc-real-join.csv", index=False)

In [190]:
# Mesclar os dataframes usando a coluna "HoraData"

df_merge = pd.merge(bimtra, tcreal, on='HoraData', how='left')

# Preencher NaN na coluna de espera com 0 (sem espera)
df_merge['trocareal'] = 1
df_merge = df_merge.groupby("flightid").agg({"trocareal": "sum"})
bimtra = bimtra.merge(df_merge.reset_index(), on="flightid", how="left")

In [191]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraDataDest,...,Visibilidademean_y,Visibilidademin_y,Visibilidademax_y,Visibilidadestd_y,Pressãomean_y,Pressãomin_y,Pressãomax_y,Pressãostd_y,troca,trocareal
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,2023-04-15 22,...,6000.0,6000.0,6000.0,NaN,1014.0,1014.0,1014.0,NaN,12,3
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,2023-04-15 22,...,6000.0,6000.0,6000.0,NaN,1014.0,1014.0,1014.0,NaN,12,3
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,2023-04-15 22,...,6000.0,6000.0,6000.0,NaN,1014.0,1014.0,1014.0,NaN,12,3
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 14,...,10000.0,10000.0,10000.0,NaN,1013.0,1013.0,1013.0,NaN,12,1


In [192]:

tcreal_hora = tcreal.groupby(["aero", "hora"]).agg({"aero": ["count"]})
tcreal_sem = tcreal.groupby(["aero", "diaSemana"]).agg({"aero": ["count"]})
tcreal_aero = tcreal.groupby(["aero"]).agg({"aero": ["count"]})

In [193]:
tcreal_aero

,aero
,count
aero,
BR,1153
CF,912
CT,1509
FL,6151
GL,2491
GR,814
KP,2060
PA,910


In [194]:
tcprev = pd.read_csv("data/tc-prev.csv")

In [195]:
bimtra["aero"] = bimtra["origem"].str[2:]

In [196]:
len(bimtra)

262416

In [197]:
tcprev_hora.reset_index(inplace=True)

In [198]:
tcprev_hora.columns = tcprev_hora.columns.droplevel(0)

In [199]:
tcprev_hora.columns = ["aero", "hora", "troca_prev_mean", "troca_prev_sum"]
tcprev_aero.reset_index(inplace=True)
tcprev_aero.columns = tcprev_aero.columns.droplevel(0)
tcprev_aero.columns = ["aero", "troca_prev_mean", "troca_prev_sum"]
tcprev_sem.reset_index(inplace=True)
tcprev_sem.columns = tcprev_sem.columns.droplevel(0)
tcprev_sem.columns = ["aero", "diaSemana", "troca_prev_mean", "troca_prev_sum"]
tcreal_hora.reset_index(inplace=True)
tcreal_hora.columns = tcreal_hora.columns.droplevel(0)
tcreal_hora.columns = ["aero", "hora", "troca_real"]
tcreal_aero.reset_index(inplace=True)
tcreal_aero.columns = tcreal_aero.columns.droplevel(0)
tcreal_aero.columns = ["aero", "troca_real"]
tcreal_sem.reset_index(inplace=True)
tcreal_sem.columns = tcreal_sem.columns.droplevel(0)
tcreal_sem.columns = ["aero", "diaSemana", "troca_real"]


In [200]:
bimtra.columns

Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'target', 'linha',
       'diaSemana', 'hora', 'HoraDataDest', 'HoraData', 'esperas', 'aero_x',
       'mean_x', 'sum_x', 'aero_y', 'mean_y', 'sum_y', 'mean', 'sum',
       'Temperaturamean_x', 'Temperaturamin_x', 'Temperaturamax_x',
       'Temperaturastd_x', 'Ponto de Orvalhomean_x', 'Ponto de Orvalhomin_x',
       'Ponto de Orvalhomax_x', 'Ponto de Orvalhostd_x',
       'Velocidade do Ventomean_x', 'Velocidade do Ventomin_x',
       'Velocidade do Ventomax_x', 'Velocidade do Ventostd_x',
       'Direção do Ventomean_x', 'Direção do Ventomin_x',
       'Direção do Ventomax_x', 'Direção do Ventostd_x', 'Visibilidademean_x',
       'Visibilidademin_x', 'Visibilidademax_x', 'Visibilidadestd_x',
       'Pressãomean_x', 'Pressãomin_x', 'Pressãomax_x', 'Pressãostd_x',
       'Temperaturamean_y', 'Temperaturamin_y', 'Temperaturamax_y',
       'Temperaturastd_y', 'Ponto de Orvalhomean_y', 'Ponto de Orvalhomin_y',
       'Ponto de Or

In [201]:
tcprev_hora.to_csv("data/tcprev_hora.csv", index=False)
tcprev_sem.to_csv("data/tcprev_sem.csv", index=False)
tcprev_aero.to_csv("data/tcprev_aero.csv", index=False)
tcreal_hora.to_csv("data/tcreal_hora.csv", index=False)
tcreal_sem.to_csv("data/tcreal_sem.csv", index=False)
tcreal_aero.to_csv("data/tcreal_aero.csv", index=False)

In [202]:
bimtra.shape

(262416, 71)

In [203]:
tcprev_sem

,aero,diaSemana,troca_prev_mean,troca_prev_sum
0,BR,0,0.221014,244
1,BR,1,0.269022,297
2,BR,2,0.293478,324
3,BR,3,0.305851,345
4,BR,4,0.268617,303
...,...,...,...,...
79,SV,2,0.293478,324
80,SV,3,0.305851,345
81,SV,4,0.268617,303
82,SV,5,0.303442,335


In [204]:
bimtra = bimtra.merge(tcprev_hora, on=["aero", "hora"], how="left")
bimtra = bimtra.merge(tcprev_sem, on=["aero", "diaSemana"], how="left")
bimtra = bimtra.merge(tcprev_aero, on=["aero"], how="left")
bimtra = bimtra.merge(tcreal_hora, on=["aero", "hora"], how="left")
bimtra = bimtra.merge(tcreal_sem, on=["aero", "diaSemana"], how="left")
bimtra = bimtra.merge(tcreal_aero, on=["aero"], how="left")
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraDataDest,...,aero,troca_prev_mean_x,troca_prev_sum_x,troca_prev_mean_y,troca_prev_sum_y,troca_prev_mean,troca_prev_sum,troca_real_x,troca_real_y,troca_real
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 01,...,KP,0.160494,52,0.293478,324,0.272119,2116,23.0,288,2060
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 01,...,GR,0.160494,52,0.293478,324,0.272119,2116,18.0,134,814
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 01,...,BR,0.160494,52,0.293478,324,0.272119,2116,35.0,210,1153
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 01,...,KP,0.160494,52,0.293478,324,0.272119,2116,23.0,288,2060
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 02,...,CF,0.160494,52,0.293478,324,0.272119,2116,25.0,109,912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,2023-04-15 22,...,GR,0.259259,84,0.303442,335,0.272119,2116,85.0,96,814
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,2023-04-15 22,...,BR,0.259259,84,0.303442,335,0.272119,2116,76.0,130,1153
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,2023-04-15 22,...,FL,0.259259,84,0.303442,335,0.272119,2116,340.0,850,6151
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 14,...,SV,0.351852,114,0.303442,335,0.272119,2116,590.0,1348,11140


In [205]:
bimtra["hora"]

0          0
1          0
2          0
3          0
4          1
          ..
262411    21
262412    21
262413    21
262414    12
262415    13
Name: hora, Length: 262416, dtype: int32

In [206]:
train = bimtra.merge(resumo, right_on="linha_first", left_on="linha", how="left")

In [207]:
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,575.6958980433018,15.414484379289922,113.11099040127081,96.26012004571069,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,1061.908684010797,13.184672048521254,113.58039360634044,84.08358708763086,13.509180931157234
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,1282.3324062624968,15.232888859932347,115.89589711595092,79.03449466219763,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,885.994544233408,13.252080132619195,122.03779859280792,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,1426.4684886193365,17.341259072228155,102.26689170812047,68.5258508294501,16.681647582814303
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,2277.3825023041554,8.22824879734405,99.87921093160097,62.36285594137942,8.362795726130804
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,643.117568170169,15.707041969914366,127.34337283196844,97.04344098509462,14.56471447373063
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,1225.918106541467,12.702016763699218,123.87140594500069,82.84562463100339,13.215119734561947


In [208]:
#esperas_semana.to_csv("data/esperas_semana.csv", index=False)
#esperas_hora.to_csv("data/esperas_hora.csv", index=False)

In [209]:
esperas

,esperas,hora,aero,dt_dep,diaSemana,HoraData
0,0,0,SBBR,2022-06-01 00:00:00,2,2022-06-01 00
1,0,1,SBBR,2022-06-01 01:00:00,2,2022-06-01 01
2,0,2,SBBR,2022-06-01 02:00:00,2,2022-06-01 02
3,0,3,SBBR,2022-06-01 03:00:00,2,2022-06-01 03
4,0,4,SBBR,2022-06-01 04:00:00,2,2022-06-01 04
...,...,...,...,...,...,...
93307,0,19,SBSV,2023-04-15 19:00:00,5,2023-04-15 19
93308,0,20,SBSV,2023-04-15 20:00:00,5,2023-04-15 20
93309,0,21,SBSV,2023-04-15 21:00:00,5,2023-04-15 21
93310,0,22,SBSV,2023-04-15 22:00:00,5,2023-04-15 22


In [210]:
train["origem"], origem = pd.factorize(train["origem"]) #verificar se rodando novamente muda de código
train["destino"], destino = pd.factorize(train["destino"])


In [211]:
train["linha"], linha = pd.factorize(train["linha"])


In [212]:
pd.Series(origem).to_csv("data/origem.csv", index=False)
pd.Series(destino).to_csv("data/destino.csv", index=False)
pd.Series(linha).to_csv("data/linha.csv", index=False)

In [213]:
del train["linha_first"]


In [214]:
del train["dt_dep"], train["dt_arr"], train["aero_x"]

In [215]:
del train["aero_y"]

In [216]:
del train["aero"]

In [217]:
train

,flightid,origem,destino,target,linha,diaSemana,hora,HoraDataDest,HoraData,esperas,...,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,2,0,2022-06-01 01,2022-06-01 00,0,...,4.0,7565.0,534.9600099520899,155.34488648758892,278.42392260334213,405.4697741951029,10.830585205248745,94.2891902188512,95.58259843029427,16.348628987169008
1,c7c5c10716335b048f86d8c52fcba3f2,1,0,2152,1,2,0,2022-06-01 01,2022-06-01 00,0,...,0.0,8344.0,507.80299655166294,132.86727827511172,248.43219881145328,343.31690044236615,9.681570140183256,81.36735891236358,91.23939542218058,16.3842888246202
2,3162de7203a972f071d5a48e8f0f4828,2,0,4613,2,2,0,2022-06-01 01,2022-06-01 00,0,...,4.0,9237.0,613.391446111362,261.70366305291907,380.86252211271164,909.9588183553077,14.824040870084836,130.64907735272433,95.29243706520055,17.765934928708283
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,1,2581,3,2,0,2022-06-01 01,2022-06-01 00,0,...,0.0,9063.0,531.1920617185621,196.2949962157983,320.12778977681785,487.7334868130487,15.784746321116943,101.97923230219794,94.35454258548637,17.81165593918845
4,504a62621cd231d6ab67e674ce538cd3,3,2,5547,4,2,1,2022-06-01 02,2022-06-01 01,1,...,1801.0,7383.0,455.0996251327685,260.85033157991717,349.6533333333333,997.4305556799677,14.632191848997632,112.58105459451855,79.57274064546145,13.37061541038086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,1,0,2138,1,5,21,2023-04-15 22,2023-04-15 21,0,...,0.0,8344.0,507.80299655166294,132.86727827511172,248.43219881145328,343.31690044236615,9.681570140183256,81.36735891236358,91.23939542218058,16.3842888246202
262412,c1e1fcbfd1d1c67c01da384ad104c0df,2,0,4741,2,5,21,2023-04-15 22,2023-04-15 21,0,...,4.0,9237.0,613.391446111362,261.70366305291907,380.86252211271164,909.9588183553077,14.824040870084836,130.64907735272433,95.29243706520055,17.765934928708283
262413,612af5b2b7cc48161bee87965785282b,6,0,3669,37,5,21,2023-04-15 22,2023-04-15 21,0,...,0.0,9423.0,569.0848511802199,241.22151471637108,378.83836858006043,751.804285672847,12.110501024534424,133.07026165110324,109.17497703457559,15.306431860431019
262414,5efa9c4eb13e214f29a73a82cf7822e4,10,0,7192,38,5,12,2023-04-15 14,2023-04-15 12,0,...,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,1225.918106541467,12.702016763699218,123.87140594500069,82.84562463100339,13.215119734561947


In [218]:
flightThrought = pd.read_csv("data/flightThourghtArea2.csv")

In [219]:
len(train)

262416

In [220]:
train = train.merge(flightThrought, on="flightid", how="left")
train.columns = [re.sub('[\[\]<>\'"\'.,]', '', col) for col in train.columns]

True

In [111]:


train.to_csv("data/train_with_flight_v2.csv", index=False)

In [113]:
for col in train.columns:
    print(col)

flightid
origem
destino
target
linha
diaSemana
hora
HoraDataDest
HoraData
esperas
mean_x
sum_x
mean_y
sum_y
mean
sum
Temperaturamean_x
Temperaturamin_x
Temperaturamax_x
Temperaturastd_x
Ponto de Orvalhomean_x
Ponto de Orvalhomin_x
Ponto de Orvalhomax_x
Ponto de Orvalhostd_x
Velocidade do Ventomean_x
Velocidade do Ventomin_x
Velocidade do Ventomax_x
Velocidade do Ventostd_x
Direção do Ventomean_x
Direção do Ventomin_x
Direção do Ventomax_x
Direção do Ventostd_x
Visibilidademean_x
Visibilidademin_x
Visibilidademax_x
Visibilidadestd_x
Pressãomean_x
Pressãomin_x
Pressãomax_x
Pressãostd_x
Temperaturamean_y
Temperaturamin_y
Temperaturamax_y
Temperaturastd_y
Ponto de Orvalhomean_y
Ponto de Orvalhomin_y
Ponto de Orvalhomax_y
Ponto de Orvalhostd_y
Velocidade do Ventomean_y
Velocidade do Ventomin_y
Velocidade do Ventomax_y
Velocidade do Ventostd_y
Direção do Ventomean_y
Direção do Ventomin_y
Direção do Ventomax_y
Direção do Ventostd_y
Visibilidademean_y
Visibilidademin_y
Visibilidademax_y
Visibi